# Importing Modules 

In [74]:
import requests
import csv
import time
from selenium import webdriver
from datetime import datetime
import pandas as pd

In [90]:
# API endpoint and headers
BASE_URL = "https://api.rhenti.com/properties"
LISTING_URL_TEMPLATE = "https://mynd.rhenti.com/#/property/{}"


HEADERS = {
         "User-Agent": "Mozilla/5.0"
}
#Params (can be adjusted)
PARAMS = {
    
    'bottomLeftCorner': [
        '-84.79722312265623',
        '33.481670043402914',
    ],
    'page': '1',
    'topRightCorner': [
        '-84.19709494882811',
        '34.01548998578539',
    ],
    'whitelabel': '65b2823e907',
    'perPage':'1000'
}
   



In [92]:
all_listings = []
page = 1

while True:
    print(f"Scraping page {page}...")
    PARAMS["page"] = str(page)  

    try:
        response = requests.get(BASE_URL, headers=HEADERS, params=PARAMS)
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        break

    if response.status_code != 200:
        print(f"Error {response.status_code} — stopping.")
        break

    data = response.json()
    listings = data

    if not listings:
        print("No more listings found.")
        break

    for listing in listings:
        listing_id = listing.get("_id")
        all_listings.append({
            "URL": LISTING_URL_TEMPLATE.format(listing_id) if listing_id else None,
            "Price": listing.get("price"),
            "Address": listing.get("full_address"),
            "Bedrooms": listing.get("bedrooms"),
            "Bathrooms": listing.get("bathrooms"),
            "Virtual_tour": listing.get("virtual_tour"),
        })

    page += 1
    time.sleep(1)




Scraping page 1...
Scraping page 2...
No more listings found.


In [94]:
# Output to CSV and Xlsx
if all_listings:
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = f"rhenti_listings_{date_str}.csv"
    with open(filename, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=all_listings[0].keys())
        writer.writeheader()
        writer.writerows(all_listings)
    print(f"Saved {len(all_listings)} listings to {filename}")
    filename = f"rhenti_{date_str}.xlsx"

    # Save to XLSX
    xlsx_filename = f"rhenti_{date_str}.xlsx"
    df = pd.DataFrame(all_listings)
    df.to_excel(xlsx_filename, index=False)
    print(f"✅ Also saved data to {xlsx_filename}")
else:
    print("No listings collected.")

Saved 59 listings to rhenti_listings_2025-06-13.csv
✅ Also saved data to rhenti_2025-06-13.xlsx
